I'm new to RL, so a tutorial Lux AI RL code was a little complicated for me.  
That's why I coded a light robot RL on my own simple env like Lux AI.  
Phase1: reward for digging target, closing to target and pickup power.  
Phase2: reward only for digging target.  


In [ ]:
!pip install --upgrade luxai_s2
!pip install pettingzoo==1.12.0 gym==0.21.0 stable-baselines3
!pip install --upgrade "importlib_metadata<5.0"


In [ ]:
%%writefile /opt/conda/lib/python3.7/site-packages/luxai_s2/version.py
__version__ = ""
# this code above is used for Kaggle Notebooks
# You might not need to run this but if you get an attribute error about the gym package, run it

In [ ]:
import importlib
import importlib_metadata
# kaggle has 6.0.0 installed but we need version <5.0
#importlib.reload(importlib_metadata) # commented out

In [ ]:
!mkdir logs

In [ ]:
from luxai_s2.env import LuxAI_S2
luxenv = LuxAI_S2() # create the environment object
lux_cfg = luxenv.state.env_cfg

In [ ]:
print(lux_cfg)

In [ ]:
import gym
import numpy as np
import cv2
import matplotlib.pyplot as plt

# actions index
ACT_STAY = 0
ACT_MOVE_L = 1
ACT_MOVE_R = 2
ACT_MOVE_U = 3
ACT_MOVE_B = 4
ACT_DIG = 5
ACT_PICKUP_POWER = 6
ACTION_NUM = 7

robo_cfg = lux_cfg.ROBOTS['LIGHT'] # for Light robot

class LightEnv(gym.Env):
    def __init__(self, saveVideo=False):
        self.saveVideo = saveVideo
        self.MAP_SIZE = lux_cfg.map_size
        self.LEARN_MAP_SIZE = self.MAP_SIZE # change at each learning step
        self.MAX_STEP = 200
        self.LOW_POWER = 30
        self.LEARN_PHASE = 1

        self.WINDOW_SIZE = int(lux_cfg.map_size * 8)

        # Actions
        self.action_space = gym.spaces.Discrete(ACTION_NUM)
        self.MOVE_MAP = np.array([[0, 0], [-1, 0], [1, 0], [0, -1], [0, 1]]) 
        
        # Observation
        #               [TargetX,          TargetY,          FactoryX,       FactoryY,       Power,                     OnFactory, TargetVal]
        LOW = np.array( [-self.MAP_SIZE, -self.MAP_SIZE, -self.MAP_SIZE, -self.MAP_SIZE, 0,                         0, 0])
        HIGH = np.array([self.MAP_SIZE,  self.MAP_SIZE,  self.MAP_SIZE,  self.MAP_SIZE,  robo_cfg.BATTERY_CAPACITY, 1, 100])
        self.observation_space = gym.spaces.Box(low=LOW, high=HIGH)

        self.reset()

    def setRandomTargetPos(self):
        tx = min(max(self.factory_position[0] + np.random.randint(-self.LEARN_MAP_SIZE, self.LEARN_MAP_SIZE),0),self.MAP_SIZE-1)
        ty = min(max(self.factory_position[1] + np.random.randint(-self.LEARN_MAP_SIZE, self.LEARN_MAP_SIZE),0),self.MAP_SIZE-1)   
        self.target_position = np.array([tx, ty])
        while np.allclose(self.target_position, self.factory_position):
            tx = min(max(self.factory_position[0] + np.random.randint(-self.LEARN_MAP_SIZE, self.LEARN_MAP_SIZE),0),self.MAP_SIZE-1)
            ty = min(max(self.factory_position[1] + np.random.randint(-self.LEARN_MAP_SIZE, self.LEARN_MAP_SIZE),0),self.MAP_SIZE-1)
            self.target_position = np.array([tx, ty])
            
    def reset(self):
        if self.saveVideo:
            video_name = f"render{self.LEARN_PHASE}.webm"
            fourcc = cv2.VideoWriter_fourcc(*'VP90')

            self.video = cv2.VideoWriter(video_name, fourcc, 20.0, (self.WINDOW_SIZE,self.WINDOW_SIZE))
        
        self.count = 0
        self.factory_position = np.array([np.random.randint(0, self.MAP_SIZE), np.random.randint(0, self.MAP_SIZE)])
        self.setRandomTargetPos()
            
        self.targetVal = 30
        self.preTargetVal = 30

        self.robo_position = self.factory_position

        self.power = robo_cfg.INIT_POWER
        self.prePower = self.power

        # Create Observation
        vec_tgt = self.robo_position - self.target_position
        vec_fact = self.robo_position - self.target_position
        observation = np.array([vec_tgt[0], vec_tgt[1],vec_fact[0],vec_fact[1],self.power,1,self.targetVal])
        self.pre_target_distance = np.sum(np.abs(vec_tgt))
        self.pre_fact_distance = np.sum(np.abs(vec_fact))
        
        self.last_act = ACT_STAY
        self.done = False
        return observation

    def act_move(self, action_index):
        action = self.MOVE_MAP[action_index]
        if self.power >= 3:
            self.robo_position = self.robo_position+action
            self.robo_position = np.clip(self.robo_position, 0, self.MAP_SIZE-1)
            self.power -= 3

    def step(self, action_index):        
        # Simulate MOVE Action
        if ACT_MOVE_L<=action_index<= ACT_MOVE_B:
            self.act_move(action_index)
        
        # Create the values for my work.
        vec_tgt = self.robo_position - self.target_position
        vec_fact = self.robo_position - self.factory_position
        distance_target = np.sum(np.abs(vec_tgt))
        distance_fact = np.sum(np.abs(vec_fact))
        onTarget = (distance_target == 0)
        onFactory = (distance_fact <= 1)

        # Simulate DIG Action
        if action_index == ACT_DIG:
            if self.power >= robo_cfg.DIG_COST and onTarget:
                self.power -= robo_cfg.DIG_COST
                self.targetVal -= robo_cfg.DIG_RUBBLE_REMOVED
                if self.targetVal <= 0:
                    self.targetVal = 0
                    
        # Simulate PICKUP Action
        if action_index == ACT_PICKUP_POWER:
            if onFactory:
                self.power = 150
        
        # Simulate Recharge
        if (self.count % 50) < 30:
            self.power += robo_cfg.CHARGE
            if self.power > robo_cfg.BATTERY_CAPACITY:
                self.power = robo_cfg.BATTERY_CAPACITY
        if self.power < 0:
            self.power = 0
        
        # Create observation
        observation = np.array([vec_tgt[0], vec_tgt[1], vec_fact[0], vec_fact[1], self.power, 1 if onFactory else 0, self.targetVal])
        reward = 0

        # Calculate Reward
        if self.LEARN_PHASE <= 1:
            reward += (self.pre_target_distance - distance_target)*2 # Reward if the robot close to target.

        reward += (self.preTargetVal - self.targetVal) * 3 # Reward if dig success

        if self.power < self.LOW_POWER:
            add_reward = self.pre_fact_distance - distance_fact  # Reward if robot close to factory when low power.
            reward += add_reward*3
            
        # Reward if robot pick up a lot of power.
        if self.LEARN_PHASE <= 1:
            diffPower = self.power - self.prePower
            if diffPower > 60:
                reward += (diffPower - 60)/5
            elif 2 < diffPower < 20:
                reward -= 1
        
        # Done over max step
        if self.count > self.MAX_STEP:
            self.done = True

        # Set parameters for next process.
        if self.targetVal <= 0: # New target
            self.setRandomTargetPos()
            vec_tgt = self.robo_position - self.target_position
            distance_target = np.sum(np.abs(vec_tgt))
            self.pre_target_distance = distance_target
            self.targetVal = 30
            
        self.last_act = action_index
        self.pre_target_distance = distance_target
        self.pre_fact_distance = distance_fact
        self.prePower = self.power
        self.preTargetVal = self.targetVal
        self.count += 1
        return observation, reward, self.done, {}

    def render(self):
        img = np.zeros((self.WINDOW_SIZE, self.WINDOW_SIZE, 3), np.uint8)
        rate = self.WINDOW_SIZE/self.MAP_SIZE
        
        if ACT_MOVE_L <= self.last_act <= ACT_MOVE_B:
            robo_color = (0,0,255)
        elif self.last_act == ACT_DIG:
            robo_color = (0,255,255)
        elif self.last_act == ACT_PICKUP_POWER:
            robo_color = (255,255,0)
        else:
            robo_color = (128,128,128)

        # Draw a factory
        cv2.rectangle(img, tuple(((self.factory_position-1)*rate).astype(np.int32)), tuple(((self.factory_position+1)*rate).astype(np.int32)), (255, 0, 0),thickness=-1)
        cv2.circle(img,  tuple((self.target_position*rate).astype(np.int32)), 3, (0, 255, 0), thickness=-1) # Draw a target
        cv2.circle(img,  tuple((self.robo_position*rate).astype(np.int32)), 3, robo_color, thickness=-1) # Draw a robot
        cv2.putText(img, f"P{self.power} T{self.targetVal}", (12,12), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 1, cv2.LINE_AA) # Draw status
        self.video.write(img)
        
        if self.done:
            self.video.release()


# Training robot with RL

In [ ]:
# Train
from stable_baselines3 import DQN
from stable_baselines3 import PPO
from stable_baselines3.common.monitor import Monitor
log_dir = "./logs"

lightEnv = LightEnv()
lightEnv.LEARN_MAP_SIZE = 6 # start from narrow size
lightEnv.MAX_STEP = 200
env = Monitor(lightEnv, log_dir, allow_early_resets=True)

model = PPO("MlpPolicy", env, verbose=0, tensorboard_log=log_dir)
# model = PPO.load("LightRobo1", env=env, verbose=0, tensorboad_log=log_dir) # Comment out and rerun this cell, if you want to try additional training.
print('start learning') 
model.learn(total_timesteps=200000, progress_bar=True, log_interval=4)
model.save("LightRobo1")
print('finish learning')

# Result of leanring

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Reading monitor.csv
df = pd.read_csv('./logs/monitor.csv', names=['r', 'l','t'])
df = df.drop(range(2)) 

# Plot rewards
x = range(len(df['r']))
y = df['r'].astype(float)
plt.plot(x, y)
plt.xlabel('episode')
plt.ylabel('reward')
plt.show()


In [ ]:
env = LightEnv(True)
env.LEARN_MAP_SIZE = 6
env.MAX_STEP = 500

model = PPO.load("LightRobo1")
obs = env.reset()


for i in range(10000):
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    env.render()
    if dones:
        break

from IPython.display import Video
Video("render1.webm")

# Phase 2 - remove inductive reward
Remove target closer reward and pickup power reward. Because they are not our essentially purpose.

In [ ]:
lightEnv.LEARN_MAP_SIZE = 7 #increase size
lightEnv.MAX_STEP = 300 #increase step
lightEnv.LEARN_PHASE = 2 #remove inductive reward
env = Monitor(lightEnv, log_dir, allow_early_resets=True)

model = PPO.load("LightRobo1", env=env, verbose=0, tensorboad_log=log_dir)
#model = PPO.load("LightRobo2", env=env, verbose=0, tensorboad_log=log_dir) # Comment out and rerun this cell, if you want to try additional training.
print('start learning') 
model.learn(total_timesteps=200000, progress_bar=True, log_interval=4)
model.save("LightRobo2")
print('finish learning')


In [ ]:
# Reading monitor.csv
df = pd.read_csv('./logs/monitor.csv', names=['r', 'l','t'])
df = df.drop(range(2)) 

# Plot rewards
x = range(len(df['r']))
y = df['r'].astype(float)
plt.plot(x, y)
plt.xlabel('episode')
plt.ylabel('reward')
plt.show()

In [ ]:
env = LightEnv(True)
env.LEARN_MAP_SIZE = 6
env.MAX_STEP = 500
env.LEARN_PHASE = 2

model = PPO.load("LightRobo2")
obs = env.reset()

for i in range(10000):
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    env.render()
    if dones:
        break

from IPython.display import Video
Video("render2.webm")

I think it learning fine.
Sometimes, the notebook view's movie might be bad learing case. Do extra train if you want see good learing case after copy.